In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import scipy.stats
from sklearn import metrics

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [13]:
# Importing the Data

data = pd.read_csv("OnlineRetail.csv")


#print(data)

In [4]:
data.groupby('InvoiceNo').head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,554065,20961,STRAWBERRY BATH SPONGE,30,22/05/2011 10:39,1.25,18287,United Kingdom
1,554065,20963,APPLE BATH SPONGE,30,22/05/2011 10:39,1.25,18287,United Kingdom
2,570715,21014,SWISS CHALET TREE DECORATION,24,12/10/2011 10:23,0.29,18287,United Kingdom
3,573167,21014,SWISS CHALET TREE DECORATION,24,28/10/2011 09:29,0.29,18287,United Kingdom
4,554065,21232,STRAWBERRY CERAMIC TRINKET BOX,12,22/05/2011 10:39,1.25,18287,United Kingdom
...,...,...,...,...,...,...,...,...
396234,562032,22371,AIRLINE BAG VINTAGE TOKYO 78,8,02/08/2011 08:48,4.25,12347,Iceland
396244,556201,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,09/06/2011 13:01,4.25,12347,Iceland
396274,556201,22725,ALARM CLOCK BAKELIKE CHOCOLATE,4,09/06/2011 13:01,3.75,12347,Iceland
396302,581180,23084,RABBIT NIGHT LIGHT,24,07/12/2011 15:52,1.79,12347,Iceland


In [3]:
transaction_series = data.groupby(['InvoiceNo'])['StockCode'].apply(list)
transaction_df = pd.DataFrame(transaction_series).rename(columns={'StockCode':'Items'})
#transaction_df

In [4]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)

# print(df)

In [5]:
frequent_itemsets = apriori(df, min_support=0.000003, use_colnames=True)

In [7]:
frequent_itemsets = apriori(df, min_support=0.000008, use_colnames=True)

In [12]:
# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.7)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(c),(e),0.000013,0.000015,0.000013,1.000000,66061.666667,0.000013,inf
1,(e),(c),0.000015,0.000013,0.000013,0.833333,66061.666667,0.000013,5.999924
2,(i),(c),0.000013,0.000013,0.000010,0.800000,63419.200000,0.000010,4.999937
3,(c),(i),0.000013,0.000013,0.000010,0.800000,63419.200000,0.000010,4.999937
4,(n),(c),0.000015,0.000013,0.000010,0.666667,52849.333333,0.000010,2.999962
...,...,...,...,...,...,...,...,...,...
87,"(c, e)","(n, i)",0.000013,0.000013,0.000010,0.800000,63419.200000,0.000010,4.999937
88,(n),"(i, c, e)",0.000015,0.000010,0.000010,0.666667,66061.666667,0.000010,2.999970
89,(i),"(n, c, e)",0.000013,0.000010,0.000010,0.800000,79274.000000,0.000010,4.999950
90,(c),"(n, i, e)",0.000013,0.000010,0.000010,0.800000,79274.000000,0.000010,4.999950
